In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from collections import defaultdict
from math import radians, cos, sin, asin, sqrt
import scipy.stats as stats
from scipy.stats import levene
import statsmodels.api as sm
import statsmodels.formula.api as smf
import folium
from folium import plugins

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/LouZhou1/ricedatathon2023/main/datathon_2024_dataset%20(1).csv")
data.head()

,Unnamed: 0,game_date,home_team,away_team,is_day_game,home_score,away_score,venue,venue_name,city,...,home_hbp,away_pa,away_1b,away_2b,away_3b,away_hr,away_fo,away_so,away_bb,away_hbp
0,0,20000329,NYN,CHN,False,3,5,TOK01,Tokyo Dome,Tokyo,...,1,38,5,1,0,1,24,4,3,0
1,1,20000330,CHN,NYN,False,1,5,TOK01,Tokyo Dome,Tokyo,...,1,44,5,0,0,0,24,9,6,0
2,2,20000403,ATL,COL,True,2,0,ATL02,Turner Field,Atlanta,...,0,32,5,0,0,2,17,6,1,1
3,3,20000403,CIN,MIL,True,3,3,CIN08,Cinergy Field,Cincinnati,...,0,20,3,1,0,1,10,4,1,0
4,4,20000403,FLO,SFN,False,6,4,MIA01,Sun Life Stadium,Miami,...,0,38,9,3,0,0,17,7,1,1


In [ ]:
data['game_date'] = data['game_date'].astype(str)
data['year'] = data['game_date'].str[:4].astype(int)
data['month'] = data['game_date'].str[4:6].astype(int)
data['day'] = data['game_date'].str[6:8].astype(int)
data.head()
data.drop(labels = ['game_date'], axis = 1)


,Unnamed: 0,home_team,away_team,is_day_game,home_score,away_score,venue,venue_name,city,state,...,away_2b,away_3b,away_hr,away_fo,away_so,away_bb,away_hbp,year,month,day
0,0,NYN,CHN,False,3,5,TOK01,Tokyo Dome,Tokyo,JAP,...,1,0,1,24,4,3,0,2000,3,29
1,1,CHN,NYN,False,1,5,TOK01,Tokyo Dome,Tokyo,JAP,...,0,0,0,24,9,6,0,2000,3,30
2,2,ATL,COL,True,2,0,ATL02,Turner Field,Atlanta,GA,...,0,0,2,17,6,1,1,2000,4,3
3,3,CIN,MIL,True,3,3,CIN08,Cinergy Field,Cincinnati,OH,...,1,0,1,10,4,1,0,2000,4,3
4,4,FLO,SFN,False,6,4,MIA01,Sun Life Stadium,Miami,FL,...,3,0,0,17,7,1,1,2000,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56770,56770,CHA,SDN,True,1,2,CHI12,Guaranteed Rate Field;U.S. Cellular Field,Chicago,IL,...,0,0,0,17,16,3,1,2023,10,1
56771,56771,DET,CLE,True,5,2,DET05,Comerica Park,Detroit,MI,...,4,0,1,16,7,6,0,2023,10,1
56772,56772,KCA,NYA,True,5,2,KAN06,Kauffman Stadium,Kansas City,MO,...,0,1,3,15,8,1,0,2023,10,1
56773,56773,SEA,TEX,True,1,0,SEA03,Safeco Field,Seattle,WA,...,0,0,0,14,10,1,1,2023,10,1


In [ ]:
sorted_city_venues_frequency = data['city'].value_counts().sort_values(ascending = True)
print(sorted_city_venues_frequency)


Omaha                  1
Fort Bragg             1
Sydney                 2
London                 2
Dyersville             2
Williamsport           5
Lake Buena Vista       6
Monterrey              7
Tokyo                 10
Dunedin               21
Buffalo               49
San Juan              49
Montreal             361
Washington          1489
Toronto             1815
Miami               1875
St. Petersburg      1885
Houston             1885
Cleveland           1886
Oakland             1888
Detroit             1889
Baltimore           1890
St. Louis           1890
Cincinnati          1890
Kansas City         1890
Pittsburgh          1890
Phoenix             1891
Atlanta             1891
Boston              1892
Seattle             1892
San Diego           1892
Minneapolis         1893
Los Angeles         1894
Denver              1894
Anaheim             1894
Arlington           1895
Philadelphia        1896
San Francisco       1898
Milwaukee           1900
New York            3785


In [ ]:
data['time_zone'] = data['city'].map(timezone_dict)

In [ ]:
latitude_dict = {'Omaha': 41.2565,'Fort Bragg': 35.1378,'Sydney': -33.8688,'London': 51.5074, 'Dyersville': 42.4842,'Williamsport': 41.2412, 'Lake Buena Vista': 28.3852,'Monterrey': 25.4380,'Tokyo': 35.6895,'Dunedin': 28.0197,'Buffalo': 42.8864,'San Juan': 18.4655,'Montreal': 45.5017,'Washington': 38.8951,'Toronto': 43.6532,'Miami': 25.7617,'St. Petersburg': 27.7676,'Houston': 29.7604, 'Cleveland': 41.4993,'Oakland': 37.8049,'Detroit': 42.3314,'Baltimore': 39.2904,'St. Louis': 38.6270,'Cincinnati': 39.1031,'Kansas City': 39.0997,'Pittsburgh': 40.4406,'Phoenix': 33.4484,'Atlanta': 33.7490,'Boston': 42.3601,'Seattle': 47.6062,'San Diego': 32.7157,'Minneapolis': 44.9778,'Los Angeles': 34.0522,'Denver': 39.7392,'Anaheim': 33.8366,'Arlington': 32.7357,'Philadelphia': 39.9526,'San Francisco': 37.7749,'Milwaukee': 43.0389,'New York': 40.7128,'Chicago': 41.8781}
longitude_dict = {'Omaha': -95.9345,'Fort Bragg': -79.0078,'Sydney': 151.2093,'London': -0.1276,'Dyersville': -91.1290,'Williamsport': -77.0011,'Lake Buena Vista': -81.5639,'Monterrey': -100.9737,'Tokyo': 139.6917,'Dunedin': -82.7718,'Buffalo': -78.8784,'San Juan': -66.1057,'Montreal': -73.5673,'Washington': -77.0369,'Toronto': -79.3832,'Miami': -80.1918,'St. Petersburg': -82.6403,'Houston': -95.3698,'Cleveland': -81.6944,'Oakland': -122.2711,'Detroit': -83.0458,'Baltimore': -76.6122,'St. Louis': -90.1994,'Cincinnati': -84.5120,'Kansas City': -94.5786,'Pittsburgh': -79.9959,'Phoenix': -112.0740,'Atlanta': -84.3880,'Boston': -71.0589,'Seattle': -122.3321,'San Diego': -117.1611,'Minneapolis': -93.2650,'Los Angeles': -118.2437,'Denver': -104.9903,'Anaheim': -117.9143,'Arlington': -97.1081,'Philadelphia': -75.1652,'San Francisco': -122.4194,'Milwaukee': -87.9065,'New York': -74.0060,'Chicago': -87.6298}
data['latitude'] = data['city'].map(latitude_dict)
data['longitude'] = data['city'].map(longitude_dict)

In [ ]:
def distance(lat1, lat2, lon1, lon2):
  lon1 = radians(lon1)
  lon2 = radians(lon2)
  lat1 = radians(lat1)
  lat2 = radians(lat2)
  dlon = lon2 - lon1
  dlat = lat2 - lat1
  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * asin(sqrt(a))
  r = 3956
  return(abs(c * r))

In [ ]:
null_mask = data.isnull().any(axis=1)
null_rows = data[null_mask]
null_rows
data

,Unnamed: 0,game_date,home_team,away_team,is_day_game,home_score,away_score,venue,venue_name,city,...,away_hr,away_fo,away_so,away_bb,away_hbp,year,month,day,latitude,longitude
0,0,20000329,NYN,CHN,False,3,5,TOK01,Tokyo Dome,Tokyo,...,1,24,4,3,0,2000,3,29,35.6895,139.6917
1,1,20000330,CHN,NYN,False,1,5,TOK01,Tokyo Dome,Tokyo,...,0,24,9,6,0,2000,3,30,35.6895,139.6917
2,2,20000403,ATL,COL,True,2,0,ATL02,Turner Field,Atlanta,...,2,17,6,1,1,2000,4,3,33.7490,-84.3880
3,3,20000403,CIN,MIL,True,3,3,CIN08,Cinergy Field,Cincinnati,...,1,10,4,1,0,2000,4,3,39.1031,-84.5120
4,4,20000403,FLO,SFN,False,6,4,MIA01,Sun Life Stadium,Miami,...,0,17,7,1,1,2000,4,3,25.7617,-80.1918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56770,56770,20231001,CHA,SDN,True,1,2,CHI12,Guaranteed Rate Field;U.S. Cellular Field,Chicago,...,0,17,16,3,1,2023,10,1,41.8781,-87.6298
56771,56771,20231001,DET,CLE,True,5,2,DET05,Comerica Park,Detroit,...,1,16,7,6,0,2023,10,1,42.3314,-83.0458
56772,56772,20231001,KCA,NYA,True,5,2,KAN06,Kauffman Stadium,Kansas City,...,3,15,8,1,0,2023,10,1,39.0997,-94.5786
56773,56773,20231001,SEA,TEX,True,1,0,SEA03,Safeco Field,Seattle,...,0,14,10,1,1,2023,10,1,47.6062,-122.3321


In [ ]:
from typing_extensions import DefaultDict
city_column = pd.Series(data["city"].unique())
api_key = 'TC3yXtAMZvXjT0hTnPklERfaQmOa7LOwuCmOg35t'
def get_timezone(city):
    api_url = 'https://api.api-ninjas.com/v1/timezone?city={}'.format(city)
    response = requests.get(api_url, headers={'X-Api-Key': api_key})
    timezone_data = response.text.split(',')
    return timezone_data[0][14:-1]
#given a column of cities, how can I create a dictionary where
timezone_dict = {}
for city in city_column:
  timezone_dict[city] = get_timezone(city)

In [ ]:
print(data.nunique())

Unnamed: 0     56775
game_date       4227
home_team         32
away_team         32
is_day_game        2
home_score        26
away_score        29
venue             56
venue_name        56
city              41
state             28
home_pa           71
home_1b           24
home_2b           12
home_3b            6
home_hr            9
home_fo           42
home_so           27
home_bb           18
home_hbp           6
away_pa           67
away_1b           22
away_2b           12
away_3b            6
away_hr            9
away_fo           41
away_so           26
away_bb           18
away_hbp           7
year              24
month              8
day               31
latitude          41
longitude         41
dtype: int64


In [ ]:
df = data[((data['home_team'] == "SEA")| (data["away_team"] == 'SEA'))  & (data['year'] == 2000)]
df.reset_index()
df_2 = data[((data['home_team'] == "MIL")| (data["away_team"] == 'MIL'))  & (data['year'] == 2000)]
df_2.reset_index()
df_2

,Unnamed: 0,game_date,home_team,away_team,is_day_game,home_score,away_score,venue,venue_name,city,...,away_fo,away_so,away_bb,away_hbp,year,month,day,latitude,longitude,time_zone
3,3,20000403,CIN,MIL,True,3,3,CIN08,Cinergy Field,Cincinnati,...,10,4,1,0,2000,4,3,39.1031,-84.5120,America/New_York
16,16,20000404,CIN,MIL,False,1,5,CIN08,Cinergy Field,Cincinnati,...,22,5,1,0,2000,4,4,39.1031,-84.5120,America/New_York
28,28,20000405,CIN,MIL,False,5,8,CIN08,Cinergy Field,Cincinnati,...,22,5,2,1,2000,4,5,39.1031,-84.5120,America/New_York
42,42,20000406,CIN,MIL,False,5,1,CIN08,Cinergy Field,Cincinnati,...,15,7,3,0,2000,4,6,39.1031,-84.5120,America/New_York
60,60,20000407,SLN,MIL,False,1,9,STL09,Busch Stadium II,St. Louis,...,15,11,6,2,2000,4,7,38.6270,-90.1994,America/Chicago
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2358,2358,20000927,MIL,CIN,False,10,6,MIL05,County Stadium,Milwaukee,...,17,7,4,1,2000,9,27,43.0389,-87.9065,America/Chicago
2373,2373,20000928,MIL,CIN,True,1,8,MIL05,County Stadium,Milwaukee,...,21,6,0,0,2000,9,28,43.0389,-87.9065,America/Chicago
2387,2387,20000929,HOU,MIL,False,3,13,HOU03,Minute Maid Park,Houston,...,23,4,6,0,2000,9,29,29.7604,-95.3698,America/Chicago
2402,2402,20000930,HOU,MIL,False,7,6,HOU03,Minute Maid Park,Houston,...,18,5,4,0,2000,9,30,29.7604,-95.3698,America/Chicago


In [ ]:
us_map = folium.Map(location=[37.7749, -122.4194], zoom_start=4)
def plot_team_travel_map(team, color, icon_color, year):
    team_df = data[((data['home_team'] == team)| (data["away_team"] == team))  & (data['year'] == year)]
    for index, row in team_df.iterrows():
        folium.Marker([row['latitude'], row['longitude']], popup=f"{row['home_team']} vs {row['away_team']} - {row['city']}", icon=folium.Icon(color=icon_color, icon='baseball', prefix='fa')).add_to(us_map)
    for i in range(1, len(team_df)):
        folium.PolyLine(locations=[(team_df.iat[i-1, 32], team_df.iat[i-1, 33]), (team_df.iat[i, 32], team_df.iat[i, 33])], color=color, dash_array='10').add_to(us_map)

#plot_team_travel_map('SEA', 'red', 'blue')
plot_team_travel_map("MIL", 'blue','green', 2000)
#plot_team_travel_map("ANA", color='green', icon_color='purple')
us_map

In [ ]:
us_map = folium.Map(location=[37.7749, -122.4194], zoom_start=4)
plot_team_travel_map('SEA', 'red', 'blue', 2011)
us_map

In [ ]:
us_map = folium.Map(location=[37.7749, -122.4194], zoom_start=4)
plot_team_travel_map('OAK', 'green', 'purple', 2023)
us_map

In [ ]:
us_map = folium.Map(location=[37.7749, -122.4194], zoom_start=4)
plot_team_travel_map('MIL', 'blue', 'green', 2011)
us_map